<a href="https://colab.research.google.com/github/Akhil-ender/Fine-Tuning-LLM-s-for-Classification/blob/main/Albert_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import sys
# Mount the Google Drive to access files stored there
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/NLP/Class_notebooks/Custom_Funtions')

# Install the latest version of torchtext library quietly without showing output
!pip install torchtext -qq
!pip install transformers evaluate wandb datasets accelerate -U -qq ## NEW LINES ##
basepath = '/content/drive/MyDrive/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# standard data science librraies for data handling and v isualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import CustomPreprocessorSpacy as cp
import torch
import torch.nn as nn


# New libraries introduced in this notebook
import evaluate
from datasets import Dataset
from torch.utils.data import  DataLoader, RandomSampler, SequentialSampler
from transformers import TrainingArguments, Trainer
from transformers import pipeline
from transformers import AlbertTokenizer, AlbertModel, AlbertConfig, AlbertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from datasets import load_dataset, load_metric
import numpy as np

import wandb

In [ ]:
model_folder = '/content/drive/MyDrive/NLP/Assignments/HW6/Albert_V2'

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/NLP/Assignments/HW6/train.csv')
test = pd.read_csv('/content/drive/MyDrive/NLP/Assignments/HW6/test.csv')

In [ ]:
X_train = train['Tweet']
X_test = test['Tweet']
y_train = train.drop(['ID','Tweet'],axis=1)
y_test = test.drop(['ID','Tweet'],axis=1)

In [ ]:
# Making Sure all the labels are an int datatype
y_train = y_train.apply(lambda x: list(map(float, x)))

In [ ]:
for column in y_test:
    y_test[column] = y_test[column].apply(lambda x: 0 if x == "NONE" else 1)
y_test = y_test.apply(lambda x: list(map(float, x)))

In [ ]:
# X_train_cleaned = cp.SpacyPreprocessor(model='en_core_web_sm', batch_size = 128).transform(X_train.values)
# X_test_cleaned = cp.SpacyPreprocessor(model='en_core_web_sm', batch_size = 128).transform(X_test.values)

In [ ]:
DS = Dataset.from_dict({
    'texts': X_train,
    'labels': y_train.values
})

testset = Dataset.from_dict({
    'texts': X_test,
    'labels': y_test.values
})

In [ ]:
# Splitting the given train set into Trainset and Evaluation Set
train_eval_split = DS.train_test_split(test_size=0.2)
trainset = train_eval_split['train']
valset = train_eval_split['test']

In [ ]:
trainset.features

{'texts': Value(dtype='string', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)}

In [ ]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
labels = y_train.columns.tolist()

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["texts"],padding="max_length", truncation=True,max_length=128)

In [ ]:
tokenized_trainsets = trainset.map(tokenize_function, batched=True)
tokenized_valsets = valset.map(tokenize_function, batched=True)
tokenized_testsets = testset.map(tokenize_function, batched=True)

Map:   0%|          | 0/6179 [00:00<?, ? examples/s]

Map:   0%|          | 0/1545 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [ ]:
config = AlbertConfig.from_pretrained('albert-base-v2', num_labels=11)

In [ ]:
config.id2label = {0:'anger', 1: 'anticipation',2: 'disgust',3: 'fear',4: 'joy',5: 'love',6: 'optimism',
  7:'pessimism', 8: 'sadness', 9: 'surprise', 10: 'trust'}

In [ ]:
config.label2id = {v: k for k, v in config.id2label .items()}

In [ ]:
config.problem_type = "multi_label_classification"

In [ ]:
config

AlbertConfig {
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "id2label": {
    "0": "anger",
    "1": "anticipation",
    "2": "disgust",
    "3": "fear",
    "4": "joy",
    "5": "love",
    "6": "optimism",
    "7": "pessimism",
    "8": "sadness",
    "9": "surprise",
    "10": "trust"
  },
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 0,
    "anticipation": 1,
    "disgust": 2,
    "fear": 3,
    "joy": 4,
    "love": 5,
    "optimism": 6,
    "pessimism": 7,
    "sadness": 8,
    "surprise": 9,
    "trust": 10
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
 

In [ ]:
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', config=config)


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Configure training parameters
training_args = TrainingArguments(

    # Training-specific configurations
    num_train_epochs=8,
    per_device_train_batch_size=10, # Number of samples per training batch
    per_device_eval_batch_size=10, # Number of samples per validation batch
    weight_decay=0.1, # weight decay (L2 regularization)
    learning_rate=3e-5, # learning arte
    optim='adamw_torch', # optimizer
    remove_unused_columns=False, # flag to retain unused columns

    # Checkpoint saving and model evaluation settings
    output_dir=model_folder,  # Directory to save model checkpoints
    evaluation_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=50,  # Perform evaluation every 50 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=100,  # Save a model checkpoint every 100 training steps
    load_best_model_at_end=True,  # Reload the best model at the end of training
    save_total_limit=1,  # Retain only the best
    # Use 'accuracy' as the metric to determine the best model
    metric_for_best_model="f1",
    greater_is_better=True,  # A model is 'better' if its accuracy is higher


    # Experiment logging configurations
    logging_strategy='steps',
    logging_steps=50,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name='Albert_V2_Improvement',  # Experiment name for Weights & Biases
)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.sigmoid(torch.tensor(logits)) > 0.5
    # Flatten to 1D arrays to compute multi-label accuracy or F1
    # predictions_flat = predictions.flatten()
    # labels_flat = labels.flatten()
    return {
        "f1": f1_score(labels, predictions, average="macro"),
        "accuracy": accuracy_score(labels, predictions),
    }


In [ ]:
# early_stopping_callback = EarlyStoppingCallback(
#     early_stopping_patience=5,          # Number of evaluations to wait before stopping if no improvement
#     early_stopping_threshold=0.1      # Minimum improvement to qualify as an improvement
# )


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_trainsets,
    eval_dataset=tokenized_valsets,
    compute_metrics=compute_metrics,
    #callbacks=[early_stopping_callback]
)

In [ ]:
# specify the project name where the experiment will be logged
%env WANDB_PROJECT = nlp_course_spring_2024-HW6-Albert

env: WANDB_PROJECT=nlp_course_spring_2024-HW6-Albert


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,F1,Accuracy
50,0.284800,0.366252,0.438359,0.232362
100,0.307300,0.348652,0.441251,0.205825
150,0.307400,0.351137,0.433313,0.211003
200,0.306000,0.354244,0.414478,0.225890
250,0.318600,0.342696,0.432192,0.227832
300,0.302600,0.337335,0.437667,0.212945
350,0.309600,0.336510,0.473561,0.242718
400,0.302500,0.335281,0.464030,0.233010
450,0.297600,0.329836,0.461167,0.234951
500,0.293300,0.334166,0.435087,0.234951


TrainOutput(global_step=4944, training_loss=0.1841573192945962, metrics={'train_runtime': 2382.1184, 'train_samples_per_second': 20.751, 'train_steps_per_second': 2.075, 'total_flos': 295595488843776.0, 'train_loss': 0.1841573192945962, 'epoch': 8.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.32048794627189636,
 'eval_f1': 0.5557973868940173,
 'eval_accuracy': 0.2537216828478964,
 'eval_runtime': 12.7023,
 'eval_samples_per_second': 121.631,
 'eval_steps_per_second': 12.202,
 'epoch': 8.0}

In [ ]:
eval_results = trainer.evaluate(tokenized_valsets)

In [ ]:
eval_results

{'eval_loss': 0.32048794627189636,
 'eval_f1': 0.5557973868940173,
 'eval_accuracy': 0.2537216828478964,
 'eval_runtime': 13.0758,
 'eval_samples_per_second': 118.157,
 'eval_steps_per_second': 11.854,
 'epoch': 8.0}

In [ ]:
#Log results to Wandb
wandb.log({"eval_accuracy": eval_results["eval_accuracy"], "eval_loss": eval_results["eval_loss"], "eval_f1": eval_results["eval_f1"]})

In [ ]:
# After training, let us check the best checkpoint
# We need this for Inference
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")

The best model was saved at step 1900.


In [ ]:
model_folder

'/content/drive/MyDrive/NLP/Assignments/HW6/Albert_V2'

In [ ]:
# Loading the saved checkpoint model
model_checkpoint = model_folder+'/checkpoint-'+best_model_checkpoint_step

In [ ]:
model_checkpoint

'/content/drive/MyDrive/NLP/Assignments/HW6/Albert_V2/checkpoint-1900'

In [ ]:
model = AlbertForSequenceClassification.from_pretrained(model_checkpoint)
model.to('cuda')
model.eval()

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    report_to=[]
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_testsets,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [ ]:
test_results = trainer.evaluate()

In [ ]:
test_results

{'eval_loss': 0.49512991309165955,
 'eval_f1': 0.0,
 'eval_accuracy': 0.048174286590978825,
 'eval_runtime': 27.882,
 'eval_samples_per_second': 116.885,
 'eval_steps_per_second': 14.633}

In [ ]:
wandb.log({"test_accuracy": test_results["eval_accuracy"], "test_loss": test_results["eval_loss"], "test_f1": test_results["eval_f1"]})

In [ ]:
wandb.finish()

eval/accuracy,▄▁▁▄▄▅▆▄█▇▅▃█▅▅▆▄▄▅██▅▇▇▅▇█▅▇▇▄▄▅▆▆▅▅▆▆▇
eval/f1,▁▁▁▃▂▃▄▄▄▅▅▆▆▆▅▇▇▇▆▇▇▆▆█▆▇▆█▇▇▇▇▆▇▇▇▇▇▇▇
eval/loss,▇▅▃▃▅▂▂▄▂▂▃▃▂▃▃▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▆▇▇▇▇▇███▁
eval/runtime,▆▁▂▂▇▁▂▂▂▂▂▂▃▃▅▃▃▃▂▂▃▂▃▃▃▃▅▂▃▃▃▄▄▄▄▃▄▄▄█
eval/samples_per_second,▃█▇▇▂█▇▇▇▇▇▇▆▆▄▆▆▆▇▇▆▇▆▆▆▆▄▇▆▆▆▅▅▅▅▆▅▅▅▁
eval/steps_per_second,▃█▇▇▂█▇▇▇▇▇▇▆▆▄▆▆▆▇▇▆▇▆▆▆▆▄▇▆▆▆▅▅▅▅▆▅▅▅▁
eval_accuracy,▁
eval_f1,▁
eval_loss,▁
test_accuracy,▁
test_f1,▁


## Model Inference

In [ ]:
# pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, config=config)

In [ ]:
# pipe("I am sad that, RCB has never won an IPL")

In [ ]:
# test['Tweet'][1]

In [ ]:
# pipe(test['Tweet'][1])

In [ ]:
# pipe("Food was awesome at that restaraunt")

In [ ]:
# pipe("Very sad that Dallas Cowboys have lost the game. Pain bro!")

In [ ]:
test_results = trainer.predict(tokenized_testsets)

In [ ]:
logits = test_results.predictions
probs = torch.sigmoid(torch.tensor(logits))
threshold = 0.5
predictions = (probs > threshold).int().numpy()

In [ ]:
labels = [x for x in y_train.columns if x not in ['Tweet']]

In [ ]:
y_pred = pd.DataFrame(predictions,columns=labels)
output_submission = pd.concat([test['ID'],y_pred],axis=  1)
output_submission

,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,0,0,0,1,0,0,1,0,0,0,0
1,2018-03739,1,0,1,0,0,0,0,0,0,0,0
2,2018-00385,1,0,0,0,0,0,0,0,1,0,0
3,2018-03001,0,0,0,0,1,0,0,0,0,0,0
4,2018-01988,0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3254,2018-03848,1,0,1,0,0,0,0,0,0,0,0
3255,2018-00416,0,1,0,1,1,0,0,0,0,0,0
3256,2018-03717,1,0,1,0,0,0,0,0,0,0,0
3257,2018-03504,0,0,0,0,1,1,1,0,0,0,0


In [ ]:
output_submission.to_csv('/content/drive/MyDrive/NLP/Assignments/HW6/Albert_V2/output_submission.csv', index=False)